## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/TextMining_WordEmbedding

/content/drive/MyDrive/TextMining_WordEmbedding


In [3]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [4]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [7]:
corpus[:50]

['영화 ‘반지의 제왕’을 보면 마법사 간달프는 호빗 마을에서 신기한 불꽃놀이를 보여준다',
 '폭죽으로 용을 밤하늘에 그렸는데, 이 용은 살아있는 것처럼 날아다닌다',
 '현실에서도 이런 불꽃놀이가 가능해졌다',
 '드론 덕분이다',
 '인텔은 4일(현지시간) ‘슈팅스타(Shooting Star)’라고 불리는 자사의 드론으로 ‘가짜 불꽃놀이’를 시연하는 데 성공했다고 밝혔다',
 '반도체 제조사인 인텔은 지난해부터 본격적으로 드론 시장에 뛰어 들었다',
 '이 드론은 무게가 280g으로 모터가 4개 달린 쿼드콥터(quadcopter)이다',
 '조립하는데 단 한 개의 나사도 쓰지 않았다고 한다',
 '500개의 슈팅스타가 불꽃놀이에 동원됐다',
 '이는 가장 많은 드론이 동시에 비행한 기록으로 기네스북에 등재됐다',
 '이들 드론은 자율 알고리즘이 내장돼 있기 때문에 하늘에 그릴 그림이나 글자를 지정해주면 개별 드론이 알아서 적절한 위치로 비행을 한다',
 '드론마다 따로 프로그래밍할 필요가 없다고 인텔은 설명했다',
 '12년 전 영국서 찾은 화석 분석… 초식공룡 이구아노돈 뇌조직 판명',
 '공룡의 뇌 화석(사진)이 세계 최초로 확인됐다',
 '데이비드 노먼 영국 케임브리지대 지구과학부 교수팀은 2004년 영국 서식스 주에서 발견된 화석을 분석한 결과 1억3300만 년 전에 살았던 것으로 추정되는 공룡의 뇌 조직인 것으로 밝혀졌다고 27일(현지 시간) 발표했다',
 '\u3000그동안 공룡의 뼈는 화석으로 많이 발견됐지만 뇌 부위는 부드럽고 부패가 빨리 진행돼 화석으로 남아 있는 경우가 없었다',
 '다른 화석으로 추정할 수밖에 없었던 공룡의 뇌 구조를 화석으로 직접 확인했다는 점에서 이번 발견은 의미를 가진다',
 '노먼 교수팀의 공룡 뇌가 화석으로 남아 있을 수 있었던 까닭은 공룡이 사망 직후 산소가 거의 없고 산성도가 높은 물에 머리가 잠겼기 때문이다',
 '일종의 ‘자연적 방부 처리’가 된 셈이다',
 '\u3000노먼 교수팀은 뇌 화석의 주인이 

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

In [8]:
okt = Okt()

In [11]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
  stop = ['Josa', 'Suffix', 'Punctuation', 'Foreign', 'Number']
  tokenized = []
  for i, j in okt.pos(sentence, stem = True, norm = True):
      if j in stop:
           continue
      tokenized.append(i)
  return tokenized

In [12]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

In [26]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(min_count = 0, min_alpha = 0.01)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 3808


In [27]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 10)

(149930, 168420)

In [29]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('화석')

[('반응', 0.9995635747909546),
 ('통해', 0.9995081424713135),
 ('나오다', 0.9995056390762329),
 ('지원', 0.9995051622390747),
 ('박사', 0.9994986057281494),
 ('연구소', 0.9994980096817017),
 ('때', 0.9994915723800659),
 ('모두', 0.9994902014732361),
 ('정보', 0.9994895458221436),
 ('교수', 0.9994884729385376)]

In [33]:
model_no_pretrained.wv.most_similar('공룡')

[('없다', 0.9997258186340332),
 ('때', 0.9997192025184631),
 ('않다', 0.9997188448905945),
 ('이', 0.9997164607048035),
 ('오다', 0.9997164011001587),
 ('등', 0.9997161626815796),
 ('정보', 0.99971604347229),
 ('크다', 0.99971604347229),
 ('시키다', 0.9997150897979736),
 ('되다', 0.9997147917747498)]

In [30]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size = 200, min_count = 0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 3808


In [31]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31436


In [32]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 10)

(166519, 168420)

In [34]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('연료')

[('추진제', 0.7780947089195251),
 ('핵연료', 0.7533149719238281),
 ('원자로', 0.7531772255897522),
 ('저렴하다', 0.7476286888122559),
 ('수소차', 0.7259962558746338),
 ('산화제', 0.7177837491035461),
 ('W', 0.7120917439460754),
 ('고압', 0.7064311504364014),
 ('가압', 0.7032783031463623),
 ('전지', 0.7008044719696045)]

In [35]:
model_no_pretrained.wv.most_similar('탄소')

[('때문', 0.9994256496429443),
 ('세계', 0.9994113445281982),
 ('단백질', 0.9994067549705505),
 ('상황', 0.9993917942047119),
 ('것', 0.9993874430656433),
 ('복잡하다', 0.9993852376937866),
 ('사람', 0.9993842840194702),
 ('인식', 0.999381959438324),
 ('오다', 0.999381959438324),
 ('같다', 0.9993817806243896)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명] 
사전 학습이 된 모델이 사전 학습이 되지 않은 모델보다 성능이 좋다고 판단했다. 사전 학습이 된 모델이 보다 연관성이 높은 단어들을 더 많이 제시하고 있다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [36]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

3808

In [39]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i])

In [41]:
pca = PCA(random_state=0).fit_transform(vector)
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

In [43]:
pca.shape

(3808, 200)

In [46]:
len(vector)

3808

In [48]:
# 시각화해보세요!

data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning: plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin

  DeprecationWarning,


In [49]:
tsne.shape

(3808, 2)

In [53]:
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [56]:
# t-SNE
tsne2 = TSNE(n_components = 2, random_state = 0, perplexity = 20, learning_rate = 0.05, init = 'random', n_iter = 3000).fit_transform(vector)

# 시각화
data = go.Scatter(
    x=tsne2[:,0],
    y=tsne2[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!